In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Brazilian Ecommerce Public Dataset Olist

You can find this data in <a>https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/code</a>

## Load Data

In [4]:
orders = pd.read_csv("../olist/olist_orders_dataset.csv")
orders_item= pd.read_csv("..\olist\olist_order_items_dataset.csv")
produts = pd.read_csv("..\olist\olist_products_dataset.csv")

customers = pd.read_csv("..\olist\olist_customers_dataset.csv")
geolocation = pd.read_csv("..\olist\olist_geolocation_dataset.csv")
payment = pd.read_csv("../olist/olist_order_payments_dataset.csv")
review = pd.read_csv("..\olist\olist_order_reviews_dataset.csv")

sellers = pd.read_csv("..\olist\olist_sellers_dataset.csv")
translation = pd.read_csv("..\olist\product_category_name_translation.csv")


## Data Resume

### Orders

In [11]:
orders.head(), orders.shape

(                           order_id                       customer_id  \
 0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
 1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
 2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
 3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
 4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
 
   order_status order_purchase_timestamp   order_approved_at  \
 0    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
 1    delivered      2018-07-24 20:41:37 2018-07-26 03:24:27   
 2    delivered      2018-08-08 08:38:49 2018-08-08 08:55:23   
 3    delivered      2017-11-18 19:28:06 2017-11-18 19:45:59   
 4    delivered      2018-02-13 21:18:39 2018-02-13 22:20:29   
 
   order_delivered_carrier_date order_delivered_customer_date  \
 0          2017-10-04 19:55:00           2017-10-10 21:25:13   
 1          2018-07-26 14:31:00       

In [8]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

In [20]:
orders['order_status'].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

In [34]:
def timedelta_para_dias(tdelta):
    if pd.isna(tdelta):
        return None
    return tdelta.total_seconds() / (24 * 3600)  # segundos → dias

# Calcular tempos em dias decimais
orders["tempo_compra_para_ordem_dias"] = (orders["order_approved_at"] - orders["order_purchase_timestamp"])
orders["tempo_ordem_para_envio_dias"] = (orders["order_delivered_carrier_date"] - orders["order_approved_at"])
orders["tempo_envio_para_entrega_dias"] = (orders["order_delivered_customer_date"] - orders["order_delivered_carrier_date"])
orders["tempo_entrega_para_estimada_dias"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"])

# Mostrar resultado
orders[[
    "tempo_compra_para_ordem_dias",
    "tempo_ordem_para_envio_dias",
    "tempo_envio_para_entrega_dias",
    "tempo_entrega_para_estimada_dias"
]].describe()


,tempo_compra_para_ordem_dias,tempo_ordem_para_envio_dias,tempo_envio_para_entrega_dias,tempo_entrega_para_estimada_dias
count,99281,97644,96475,96476
mean,0 days 10:25:08.739486910,2 days 19:19:15.298850927,9 days 07:55:59.254148743,-12 days +19:42:04.042383598
std,1 days 02:02:16.813098347,3 days 13:11:10.497083683,8 days 18:14:34.522067763,10 days 04:28:00.181984555
min,0 days 00:00:00,-172 days +18:44:38,-17 days +21:41:31,-147 days +23:36:47
25%,0 days 00:12:54,0 days 21:00:44,4 days 02:23:55.500000,-17 days +18:08:05.250000
50%,0 days 00:20:36,1 days 19:38:29.500000,7 days 02:23:40,-12 days +01:13:31.500000
75%,0 days 14:34:51,3 days 13:55:52.500000,12 days 00:41:55.500000,-7 days +14:38:24
max,187 days 21:10:50,125 days 18:18:06,205 days 04:35:00,188 days 23:24:07
